# Lab 05: MobileNet

Trong bài thực hành này:
- Cài đặt, train MobileNet với data MNIST


Reference:
- MobileNet V1: MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications, https://arxiv.org/abs/1704.04861

## 1. Xây dựng MobileNet bằng tf.keras.layers

Trong phần này chúng ta sẽ xây dựng và huấn luyện model ResNet-34 trên dataset MNIST (ảnh được resize)

In [ ]:
#import thư viện cần thiết
## thư viện machine learning và hỗ trợ
import tensorflow as tf
from tensorflow import keras
import numpy as np

## thư viện để vẽ đồ thị
import matplotlib.pyplot as plt


In [ ]:
mobilenet = keras.applications.mobilenet.MobileNet(input_shape=(32,32,1), 
                                                   alpha=1.0, 
                                                   depth_multiplier=1, 
                                                   dropout=1e-3, 
                                                   include_top=True, 
                                                   weights=None, 
                                                   input_tensor=None, 
                                                   pooling=None, 
                                                   classes=10)
mobilenet.summary()

### 1.1 Depthwise Separable Convolution

Trong phần này sẽ xây dựng lớp DepthwiseSeparableConvolution gồm 
- Depthwise Convolution: tích chập cho từng channel của input, kernel_size=[3,3]
- Pointwise convolution: là lớp convolution thông thường kernel size [1,1] với input từ Depthwise Convolution

<img src="DepthSeparableConvolution.png" width="40%" height="40%">

Hình trên bên phải vẽ cấu trúc của lớp Depthwise Separable Convolution

In [ ]:
## Import các layer cần thiết
from tensorflow.keras.layers import Input, Dense, DepthwiseConv2D, Convolution2D, MaxPool2D, BatchNormalization, ReLU, GlobalAveragePooling2D
from tensorflow.keras.regularizers import l2

## Định nghĩa 1 Depthwise Separable Convolution
class DepthwiseSeparableConvolution(keras.layers.Layer):

    def __init__(self, n_filters=64, l2_regularizer=0.0, down_sampling=False):
        ## Gọi hàm khởi tạo của keras.layers.Layer và lưu lại các thông số
        super(DepthwiseSeparableConvolution, self).__init__()
        self.n_filters = n_filters
        self.down_sampling = down_sampling
        self.l2_regularizer = l2_regularizer

    def build(self, input_shape):
        
        ## Nếu cần down sampling thì convolutional layer dùng strides=[2,2]
        strides = [1,1]
        if self.down_sampling:
            strides = [2,2]
        
        ##Khai báo các layer 
        self.depthwise_conv = DepthwiseConv2D(kernel_size=[3,3],
                                              strides=strides,
                                              padding='same',
                                              use_bias=False,
                                              activation=None)
        self.depthwise_batch = BatchNormalization()
        self.depthwise_relu = ReLU()

        self.pointwise_conv = Convolution2D(filters=self.n_filters,
                                            kernel_size=[1,1],
                                            strides=[1,1],
                                            padding='same',
                                            use_bias=False,
                                            kernel_regularizer=l2(self.l2_regularizer),
                                            activation=None)
        self.pointwise_batch = BatchNormalization()
        
        self.pointwise_relu = ReLU()

    def call(self, inputs):
        
        ## Thiết lập các input cho các layer đã khai báo
        x = inputs
        x = self.depthwise_conv(x)
        x = self.depthwise_batch(x)
        x = self.depthwise_relu(x)
        x = self.pointwise_conv(x)
        x = self.pointwise_batch(x)
        x = self.pointwise_relu(x)
        return x

### 1.2 Baseline MobileNet

<img src="MobileNETv1.png" width="40%" height="40%">

Hình trên vẽ các cấu trúc mạng MobileNetV1

In [ ]:

l2_regularizer_rate = 0.0

inputs = keras.layers.Input(shape=(32,32,1))

conv1 = Convolution2D(filters=32,
                      kernel_size=[3,3],
                      strides=[2,2],
                      padding='same',
                      use_bias=False,
                      kernel_regularizer=l2(l2_regularizer_rate),
                      activation=None)(inputs)

batch1 = BatchNormalization()(conv1)        

relu1 = ReLU()(batch1)


dw_pw_conv = DepthwiseSeparableConvolution(n_filters=64,
                                            l2_regularizer=l2_regularizer_rate,
                                            down_sampling=False)(relu1)

####
dw_pw_conv = DepthwiseSeparableConvolution(n_filters=128,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=True)(dw_pw_conv)

dw_pw_conv = DepthwiseSeparableConvolution(n_filters=128,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

####
dw_pw_conv = DepthwiseSeparableConvolution(n_filters=256,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=True)(dw_pw_conv)

dw_pw_conv = DepthwiseSeparableConvolution(n_filters=256,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

####
dw_pw_conv = DepthwiseSeparableConvolution(n_filters=512,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=True)(dw_pw_conv)

dw_pw_conv = DepthwiseSeparableConvolution(n_filters=512,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

dw_pw_conv = DepthwiseSeparableConvolution(n_filters=512,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

dw_pw_conv = DepthwiseSeparableConvolution(n_filters=512,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

dw_pw_conv = DepthwiseSeparableConvolution(n_filters=512,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

dw_pw_conv = DepthwiseSeparableConvolution(n_filters=512,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=False)(dw_pw_conv)

#####
dw_pw_conv = DepthwiseSeparableConvolution(n_filters=1024,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=True)(dw_pw_conv)

#####
dw_pw_conv = DepthwiseSeparableConvolution(n_filters=1024,
                                           l2_regularizer=l2_regularizer_rate,
                                           down_sampling=True)(dw_pw_conv)

avage_pool = GlobalAveragePooling2D()(dw_pw_conv)

softmax = Dense(units=10, activation='softmax')(avage_pool)

## Compile model
model = keras.models.Model(inputs=inputs, outputs=softmax)
model.compile(optimizer=keras.optimizers.Adam(), 
             loss=tf.keras.losses.sparse_categorical_crossentropy,
             metrics=["accuracy"])
    

## In toàn bộ cấu trúc của model
print("Cấu trúc của model: ")
model.summary()



### 1.3 Resize MNIST

In [ ]:

# Tải dataset MNIST từ tensorflow
## MNIST là bài toán dự đoán một ảnh thể hiện ký tự số nào

## tải MNIST dataset từ keras
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
##resacle ảnh thành ảnh thực trong đoạn [0,1]
X_train, X_test = X_train/255.0, X_test/255.0

##in dataset
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)



In [ ]:
## import thư viện OpenCV trên python
#!pip3 install opencv-python

### Thử resize một ảnh
import cv2
resized_img = cv2.resize(X_train[0], dsize=(32,32))
print("Kích thước ảnh sau resize: ", resized_img.shape)

In [ ]:
## Resize toàn bộ ảnh train tập train và test
X_train = np.array([cv2.resize(img, dsize=(32,32)) for img in X_train])
X_test = np.array([cv2.resize(img, dsize=(32,32)) for img in X_test])
print("Kích thước tập sau khi resize: ", X_train.shape, X_test.shape)

## In xem ảnh còn ổn không sau khi resize
plt.imshow(X_train[0])
plt.show()

## Reshape ảnh để phù hợp với input của model (thêm một trục)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
print("Kích thước tập sau khi reshape: ", X_train.shape, X_test.shape)

plt.imshow(X_train[0,:,:,0])
plt.show()

#Tách một phần tập train thành tập valid
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1)

## Reshape ảnh để phù hợp với input của model (thêm một trục)

### 1.4 Train

In [ ]:
# Checkpoint Callback
mc = keras.callbacks.ModelCheckpoint(filepath="mobilenet_mnist.h5", 
                                     monitor='val_loss',
                                     mode='min', 
                                     verbose=0, 
                                     save_best_only=True)

## Train  ## Khuyến cáo chạy COLAB (hoặc tương tự)
history = model.fit(X_train, y_train,
                    batch_size=100,
                    epochs=10,
                    validation_data=(X_valid, y_valid),
                    callbacks=[mc])                    

## Load lại model tốt nhất đã lưu
print("best model: ")
model.load_weights("mobilenet_mnist.h5")
valid_loss, valid_acc = model.evaluate(X_valid, y_valid)
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Valid: loss {} acc {} -- Test: loss {} valid {}".format(valid_loss, valid_acc, test_loss, test_acc))

## Bài tập

Reference: MobileNetV2: Inverted Residuals and Linear Bottlenecks, https://arxiv.org/abs/1801.04381

Bài tập là xây dựng mạng MobileNetV2 với những cải tiến dựa trên bottleneck và residual.
1. Xây dựng lớp Bottleneck DepthSeparable Convolution with Residuals:
```python
class BottleneckDepthSeparableConvolutionWithResiduals(keras.layers.Layer):

    def __init__(self, n_filters=64, expansion_factor=6, l2_regularizer=0.0, down_sampling=False):
        pass
```
2. Xây dựng mạng MobileNetV2:

<img src="MobileNETv2.png" width="40%" height="40%">

